# Muhammad Murad 21K3064 BCS-6B

# IR Assignment 1

# Loading libraries and files

In [1]:
from nltk.stem import PorterStemmer
import re
import glob

In [2]:
txt_files = glob.glob('*.txt')

print(txt_files)

['1.txt', '11.txt', '12.txt', '13.txt', '14.txt', '15.txt', '16.txt', '17.txt', '18.txt', '2.txt', '21.txt', '22.txt', '23.txt', '24.txt', '25.txt', '26.txt', '3.txt', '7.txt', '8.txt', '9.txt', 'Gold Query-Set Boolean Queries.txt', 'Stopword-List.txt']


In [3]:
exclude = ['Stopword-List.txt','Gold Query-Set Boolean Queries.txt']
documents_ = [file for file in txt_files if file not in exclude]    

In [4]:
documents = []
for file in documents_:
    with open(file, 'r') as files:
        doc_id = int(re.search(r'\d+', file).group())
        documents.append((doc_id, files.read()))

# Preprocessing

In [5]:

def load_stopwords(stopword):
    with open(stopword,'r') as file:
        stopwords = set(file.read().split())
    return stopwords


In [6]:
stopwords = load_stopwords(txt_files[-1])
print(stopwords)

{'the', 'have', 'all', 'a', 'we', 'once', 'as', 'had', 'at', 'her', 'am', 'in', 'has', 'up', 'and', 'his', 'be', 'are', 'on', 'for', 'do', 'is', 'to', 'can', 'of', 'no'}


In [7]:
stemmer = PorterStemmer()

In [8]:
def extract_doc_id(filename):
    match = re.search(r'(\d+)', filename)
    return int(match.group(1)) if match else None


In [23]:
# def tokenize_document(document_content):
#     word_pattern = r'\b(?:[a-zA-Z0-9]+(?:-[a-zA-Z0-9]+)*)\b'
#     return re.findall(word_pattern, document_content.lower())


# tried to use customized tokenization below but because of complexity and documents it makes kernel so slow 

def tokenize_document(document_content):
    word_boundaries = [' ', '\n', '\t', ',', '.', ';', ':', '?', '!', '"', "'", '(', ')', '[', ']', '{', '}', '/', '\\', '|', '=', '+', '-', '*', '&', '%', '$', '@', '<', '>', '^', '`', '~']

    words = []
    current_word = ''
    for char in document_content:
        if char in word_boundaries:
            if current_word:
                words.append(current_word.lower())  
                current_word = ''
        else:
            current_word += char
    if current_word:
        words.append(current_word.lower())  

    return words


In [24]:
def preprocessing(documents, stopwords, stemmer):
    preprocessed_words = {}
    for filename in documents:
        doc_id = extract_doc_id(filename)
        if doc_id:
            with open(filename, 'r') as file:
                document_content = file.read()
                words = tokenize_document(document_content)
                stemmed_words = [stemmer.stem(word) for word in words if word not in stopwords]
                preprocessed_words[doc_id] = stemmed_words
    return preprocessed_words


# Inverted Index and Positional Index

In [26]:
%%time
def creating_inverted_index(documents, stopwords, stemmer):
    inverted_index = {}
    preprocessed_docs = preprocessing(documents, stopwords, stemmer)
    for doc_id, words in preprocessed_docs.items():
        for word in words:
            inverted_index.setdefault(word, set()).add(doc_id)
    return inverted_index
inverted_index = creating_inverted_index(documents_, stopwords, stemmer)

Wall time: 5.35 s


In [27]:
%%time
def creating_positional_index(documents, stopwords, stemmer):
    positional_index = {}
    for doc_id, doc_content in documents:
        words = tokenize_document(doc_content)
        for position, word in enumerate(words):
            if word not in stopwords:
                stemmed_word = stemmer.stem(word)
                if stemmed_word not in positional_index:
                    positional_index[stemmed_word] = {}
                if doc_id not in positional_index[stemmed_word]:
                    positional_index[stemmed_word][doc_id] = []
                positional_index[stemmed_word][doc_id].append(position)
    return positional_index
positional_index = creating_positional_index(documents, stopwords, stemmer)


Wall time: 5.14 s


# Proximity Queries

In [28]:
def intersect_proximity(results, term_results, proximity):
    intersected_results = {}
    for doc_id, positions in term_results.items():
        if doc_id in results:
            for pos in positions:
                for prev_pos in results[doc_id]:
                    if abs(prev_pos - pos) <= proximity: # our threshold of proximity 
                        intersected_results.setdefault(doc_id, set()).add(pos)
    return intersected_results

def union_proximity(results, term_results, proximity):
    unioned_results = results.copy()
    for doc_id, positions in term_results.items():
        for pos in positions:
            unioned_results.setdefault(doc_id, set()).add(pos)
    return unioned_results

def difference_proximity(results, term_results, positional_index, proximity): 
    if not results:
        all_doc_ids = set(positional_index.keys())
        return {doc_id: set() for doc_id in all_doc_ids.difference(term_results.keys())}
    
    differed_results = results.copy() 
    for doc_id, positions in term_results.items():
        if doc_id in differed_results:
            for pos in positions:
                for prev_pos in differed_results[doc_id]:
                    if abs(prev_pos - pos) <= proximity:
                        differed_results[doc_id].remove(prev_pos)
    return differed_results



# Building Bolean Model 

In [29]:
# without proximity 
def process_boolean_query(query, inverted_index):
    terms = query.split()
    results = None
    operator = None
    
    for term in terms:
        if term in ['AND', 'OR', 'NOT']:
            operator = term
        else:
            term_results = inverted_index.get(term, set()) 
            term_results = set(int(doc_id) for doc_id in term_results) # getting doc id here sir basic idea is to and term ids
            if operator == 'AND':
                results = term_results if results is None else results.intersection(term_results)
            elif operator == 'OR':
                results = term_results if results is None else results.union(term_results) # union of their index
            elif operator == 'NOT':
                all_doc_ids = set(int(doc_id) for doc_ids in inverted_index.values() for doc_id in doc_ids)
                term_results = all_doc_ids.difference(term_results) # taking difference
                if results is None:
                    results = term_results
                else:
                    results = results.intersection(term_results)
            else:
                results = term_results
    
    return results


In [30]:
# with proximity 
def process_boolean_query_proximity(query, positional_index, proximity):
    terms = query.split()
    results = None
    operator = None
    
    for term in terms:
        if term in ['AND', 'OR', 'NOT']:
            operator = term
        else:
            term_results = positional_index.get(term, {})
            if operator == 'AND':
                results = term_results if results is None else intersect_proximity(results, term_results, proximity)
            elif operator == 'OR':
                results = term_results if results is None else union_proximity(results, term_results, proximity)
            elif operator == 'NOT':
                results = difference_proximity(results, term_results, positional_index, proximity)
            else:
                results = term_results
    
    return results



# Results 

## Simple Boolean model

In [31]:
%%time
queries = ['heart', 'heart AND attack', 'NOT model', 'model','transformer OR model', 'transformer AND model',
           'cancer AND learning', 'tubing', 'NOT classification AND NOT feature', 'feature AND selection AND classification',
           'feature AND selection AND redundancy', 'java', 'cancer OR learning', 'transformer']

for query in queries:
     results = process_boolean_query(query, inverted_index) # accepts inverted index and query 
     
     print(f"Query: {query} ->  Results: {results}")
     print()   


Query: heart ->  Results: {1, 3, 7, 8, 9, 11, 26}

Query: heart AND attack ->  Results: {8, 7}

Query: NOT model ->  Results: {11}

Query: model ->  Results: {1, 2, 3, 7, 8, 9, 12, 13, 14, 15, 16, 17, 18, 21, 22, 23, 24, 25, 26}

Query: transformer OR model ->  Results: {1, 2, 3, 7, 8, 9, 12, 13, 14, 15, 16, 17, 18, 21, 22, 23, 24, 25, 26}

Query: transformer AND model ->  Results: set()

Query: cancer AND learning ->  Results: set()

Query: tubing ->  Results: set()

Query: NOT classification AND NOT feature ->  Results: {1, 2, 3, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 18, 21, 22, 23, 24, 25, 26}

Query: feature AND selection AND classification ->  Results: set()

Query: feature AND selection AND redundancy ->  Results: set()

Query: java ->  Results: {16}

Query: cancer OR learning ->  Results: {2, 7, 8, 11, 16, 22, 24, 26}

Query: transformer ->  Results: set()

Wall time: 32.9 ms


## Boolean model with proximity

In [32]:
%%time
queries = ['heart', 'heart AND attack', 'Not Model' , 'model','transformer OR model', 'transformer AND model',
           'cancer AND learning', 'tubing', 'NOT classification AND NOT feature', 'feature AND selection AND classification',
           'feature AND selection AND redundancy', 'java', 'cancer OR learning', 'transformer']

proximity = 5
for query in queries:
     results = process_boolean_query_proximity(query, positional_index, proximity)
     
     print(f"Query: {query} -> Results: {results}")
     print()

Query: heart -> Results: {1: [9921], 11: [4, 62, 70, 84, 96, 128, 159, 199, 212, 220, 309, 321, 329, 352, 366, 415, 499, 543, 570, 669, 726, 756, 787, 790, 805, 813, 841, 868, 923, 928, 958, 1020, 1177, 1211, 1220, 1227, 1229, 1244, 1269, 1302, 1332, 1367, 1376, 1398, 1442, 1472, 1477, 1501, 1511, 1532, 1592], 26: [5899], 3: [1438], 7: [27664, 39034], 8: [8, 206, 220, 338, 434, 546, 820, 1352, 1511, 1844, 2039, 2400, 2512, 2653, 2663, 2727, 2733, 2852, 2905, 3026, 3037, 3513, 3925, 4189, 4350, 4377, 4390, 4394, 4420, 4431, 4472, 4488, 4503, 4511, 4530, 4537, 4544, 4564, 4571, 4576, 4600, 4604, 4617, 4699, 4746, 4749, 4880, 4887, 4897, 4937, 5262, 5297, 5390, 5442, 5485, 5518, 5539, 5587, 5593, 5648, 5659, 5690, 5725, 5749, 5760, 5782, 5808, 5817, 5854, 5878, 5911, 5933, 5947, 5969, 6004, 6008, 6046, 6051, 6077, 6083, 6109, 6113, 6141, 6175, 6198, 6209, 6233, 6307, 6365, 6395, 6420, 6428, 6451, 6483, 6492, 6708], 9: [4, 12, 132, 320, 352, 640, 660, 894, 1035, 1177, 1425, 2131, 2136, 239

## GUI for demonstration 

In [33]:
import tkinter as tk

def display_results(results):
    result_text.delete('1.0', tk.END)
    result_text.insert(tk.END, "Documents matching the query:\n")
    result_text.insert(tk.END, "\n".join(map(str, results)))

def process_query():
    query = query_entry.get()
    if proximity_option.get():
        proximity = int(proximity_entry.get())
        results = process_boolean_query_proximity(query, positional_index, proximity)
    else:
        results = process_boolean_query(query, inverted_index)
    display_results(results)

root = tk.Tk()
root.title("Boolean Query Processor")
 
bg_color = "#f0f0f0"
button_bg_color = "#007bff"
button_fg_color = "black"

root.configure(bg="#f9f9f9")

 
input_frame = tk.Frame(root, bg=bg_color)
input_frame.pack(pady=20)

query_label = tk.Label(input_frame, text="Enter Query:", bg=bg_color)
query_label.grid(row=0, column=0, padx=10)

query_entry = tk.Entry(input_frame, width=80)
query_entry.grid(row=0, column=1, padx=10)

proximity_label = tk.Label(input_frame, text="Enter Proximity (optional):", bg=bg_color)
proximity_label.grid(row=1, column=0, padx=10)

proximity_entry = tk.Entry(input_frame, width=20)
proximity_entry.grid(row=1, column=1, padx=10)

 
proximity_frame = tk.Frame(root, bg=bg_color)
proximity_frame.pack(pady=5)

proximity_option = tk.BooleanVar()
proximity_checkbutton = tk.Checkbutton(proximity_frame, text="Proximity Query", variable=proximity_option, bg=bg_color)
proximity_checkbutton.pack()

 
button_frame = tk.Frame(root, bg=bg_color)
button_frame.pack(pady=10)

process_button = tk.Button(button_frame, text="Process Query", command=process_query, bg=button_bg_color, fg=button_fg_color)
process_button.pack()

result_text = tk.Text(root, height=20, width=100)
result_text.pack(pady=20)

root.mainloop()


# Challenges I faced 
## Tokenization 
### Problem is that there are multiple formattings i have choosed that which gives at least better results 
### it gives multi-words doesn't tokenizes single word mostly which effects the query 
### to solve this we can use library word_tokenize but as you said sir we are not allowed
## Stemming
### if take look the porter stemmer does good but not best like if there's word challenge it stems too challeng
### like this it completely changes the words which make problem while query processing